# simulation with ivs data

### Imports
Import the required libraries

In [7]:
# package(s) related to time, space and id
import datetime, time
import platform
import random
import os
import pathlib

# you need these dependencies (you can get these from anaconda)
# package(s) related to the simulation
import simpy

# spatial libraries 
import pyproj
import shapely.geometry
from shapely.geometry import Point
import shapely
import geopandas as gpd
import movingpandas as mpd

# package(s) for data handling
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

# OpenTNSIM
import opentnsim
import opentnsim.core as core
import opentnsim.graph_module as graph_module
import opentnsim.plot as plot


# dtv_backend
import dtv_backend.fis as fis
import dtv_backend.network
import dtv_backend.network.network_utilities
import dtv_backend.postprocessing
import dtv_backend.simple
import dtv_backend.simulate

# Used for mathematical functions
import math             
import numpy as np

# Used for making the graph to visualize our problem
import networkx as nx  

### Create graph

In [8]:
#Load graph
url = "https://zenodo.org/record/6673604/files/network_digital_twin_v0.3.pickle?download=1"
graph1 = dtv_backend.fis.load_fis_network(url).copy()

# remove edge ('8865412', 'B20540_A')
graph1.remove_edge('8865412', 'B20540_A')


# #remove edges without classification
# for edge in graph1.edges:
#     if str(graph1.edges[edge]['Classification']) == 'nan':
#         graph1.remove_edge(edge[0], edge[1])

graph = graph_module.Graph()
graph.graph = graph1
graph.graph_info = opentnsim.utils.info(graph.graph)

In [9]:
afzetting = pd.DataFrame({'A': graph1.nodes['8865412'],
'B': graph1.nodes['B20540_A']}).T
afzetting = gpd.GeoDataFrame(afzetting)
afzetting.to_file('plots_routes/afzetting_locatie')

In [10]:
df_edges = nx.to_pandas_edgelist(graph1)
gpd.GeoDataFrame(df_edges).to_file('plots_routes/edges_met_afzetting')
gpd.GeoDataFrame(df_edges).dtypes

D:\Users\ALPF\AppData\Local\Temp\ipykernel_30968\2119284774.py:2: UserWarning:

Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.



source                object
target                object
GeneralDepth         float64
WidePushedLength     float64
Classification        object
SeaFairingWidth      float64
SeaFairingDepth      float64
CoupledLength        float64
subgraph               int64
Wkt                   object
length_m             float64
Name                  object
CoupledDepth         float64
PushedDepth          float64
GeneralLength        float64
length_deg           float64
StartJunctionId       object
EndJunctionId         object
PushedWidth          float64
Id_navigability      float64
Length               float64
SeaFairingLength     float64
WidePushedWidth      float64
PushedLength         float64
WidePushedDepth      float64
Code                  object
GeneralWidth         float64
geometry            geometry
length               float64
GeoType               object
GeneralHeight        float64
Description           object
SeaFairingHeight     float64
CoupledWidth         float64
dtype: object

### Make vessels and paths

##### read data

In [11]:
data = gpd.read_file("data\ivs\ivs-2024-geocoded-sample.gpkg")
data['datetime'] = pd.to_datetime(data['v05_06_begindt_evenement_iso'], format = 'ISO8601', errors = 'coerce')
data.dropna(subset = ['datetime', 'geometry'], inplace = True)


D:\Users\ALPF\AppData\Local\Temp\ipykernel_30968\2736635994.py:2: FutureWarning:

In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`



filter only on rotterdam to duisburg, 2023. Only keep 5 ships

In [12]:
data1= data

In [13]:
data = data1
data = data[
    (data.UNLO_herkomst== "NLDOR")
    #(data.UNLO_herkomst== "NLGAM")
     & (data.UNLO_bestemming.str.startswith("DE"))
     & (data.v05_06_begindt_evenement_iso.str.startswith("2023"))
     ].head()
data

,Jaarmaand,Jaar,Maand,Weeknr,v05_06_begindt_evenement_iso,v05_06_Begindt_evenement,UNLO_herkomst,UNLO_bestemming,v15_1_Scheepstype_RWS,SK_CODE,v18_Laadvermogen,v28_Beladingscode,v38_Vervoerd_gewicht,v30_4_Containers_TEU_S,nstr_nw,nst2007_nw,geometry,datetime
14694,2308,2023,8,32,2023-08-10T08:00:00+02:00,10 augustus 2023 08:00:00 uur,NLDOR,DEDUI,9,C3l,4320.0,5.0,0.0,0.0,9,16,"LINESTRING (4.63333 51.78333, 6.75000 51.46667)",2023-08-10 08:00:00+02:00
20814,2301,2023,1,1,2023-01-04T16:00:00+01:00,04 januari 2023 16:00:00 uur,NLDOR,DEMHG,9,C3l,5730.0,7.0,3550000.0,0.0,2,2.1,"LINESTRING (4.63333 51.78333, 8.45000 49.48333)",2023-01-04 16:00:00+01:00
27391,2309,2023,9,39,2023-09-25T10:00:00+02:00,25 september 2023 10:00:00 uur,NLDOR,DENSS,2,M3,690.0,5.0,0.0,0.0,9,16,"LINESTRING (4.63333 51.78333, 6.68333 51.18333)",2023-09-25 10:00:00+02:00
32727,2309,2023,9,39,2023-09-26T06:00:00+02:00,26 september 2023 06:00:00 uur,NLDOR,DEGEK,1,M2,510.0,7.0,375000.0,0.0,0,1.1,"LINESTRING (4.63333 51.78333, 7.11667 51.51667)",2023-09-26 06:00:00+02:00
35894,2308,2023,8,32,2023-08-12T15:00:00+02:00,12 augustus 2023 15:00:00 uur,NLDOR,DEFRA,21,BII-2L,5320.0,5.0,0.0,0.0,9,16,"LINESTRING (4.63333 51.78333, 8.68333 50.11667)",2023-08-12 15:00:00+02:00


##### Create vessels

In [14]:
# Make a class out of mix-ins
TransportResource = type('TransportResource', 
                         (core.Identifiable, core.ContainerDependentMovable, 
                          core.HasResource, core.Routable,
                          core.VesselProperties,
                         core.ExtraMetadata), 
                         {})

#define speed: 
def compute_v_provider(v_empty, v_full):
    return lambda x: 1

vessels = []
for index, row in data.iterrows():
    #determine path
    point_1 = fis.find_closest_node(graph.graph, Point(row.geometry.coords[0]))
    point_2 = fis.find_closest_node(graph.graph, Point(row.geometry.coords[-1]))
    path = nx.dijkstra_path(graph.graph, point_1[0], point_2[0])
    #determine capacity
    capacity = max(row.v18_Laadvermogen*1000, row.v38_Vervoerd_gewicht, 1)
    data_vessel = {"env": None,
               "name": row.name,
               "type": row['v15_1_Scheepstype_RWS'],
               "B": 1,
               "L": 10,
               "route": path,
               "geometry": Point(row.geometry.coords[0]),  # lon, lat
               "capacity": capacity,
               "v": 1,
               "compute_v": compute_v_provider(v_empty=1, v_full=1),
               }
    vessel = TransportResource(**data_vessel)
    vessels.append(vessel)

vessels = [TransportResource(**{
    "env" : None,
    "name": 'korte_route', 
    "type": 'M6',
    "B": 1, 
    "L": 10,
    "route": nx.dijkstra_path(graph.graph, "8865735", "8861687"),
    "geometry": Point(row.geometry.coords[0]),  # lon, lat
    "capacity": capacity,
    "v": 1,
    "compute_v": compute_v_provider(v_empty=1, v_full=1),
})]

### Start simulation

In [15]:
def start(env, vessel):
    while True:
        vessel.log_entry_v0("Start sailing", env.now, "", vessel.geometry)
        yield from vessel.move()
        vessel.log_entry_v0("Stop sailing", env.now, "", vessel.geometry)
        
        if vessel.geometry == nx.get_node_attributes(env.FG, "geometry")[vessel.route[-1]]:
            break

In [16]:
# Start simpy environment
simulation_start = datetime.datetime.now()
env = simpy.Environment(initial_time = time.mktime(simulation_start.timetuple()))
env.epoch = time.mktime(simulation_start.timetuple())


# Add graph to environment
graph.add_resources(list(graph.graph.edges), np.ones(len(list(graph.graph.edges))), env)
env.FG = graph.graph



In [17]:

vessels = vessels[0:3]
for i, vessel in enumerate(vessels):
    # Add environment and path to the vessel
    vessel.env = env

    # Add the movements of the vessel to the simulation
    env.process(start(env, vessel))


env.epoch = datetime.datetime.now()

env.run()

### Obtain vessel log information
The cel below uses the vessel log. The core function *log_entry* is used, which takes four arguments:

- **Log.** A text to describe what is logged.
- **t.** The timestamp.
- **Value.**  The value for the log (for sailing this is the distance).
- **Geometry** The location of the vessel while loggin.

In [18]:
vessel_log = gpd.GeoDataFrame(vessels[0].logbook, geometry='Geometry')
vessel_log = mpd.Trajectory(vessel_log, traj_id='vessel_1_trip_1', obj_id='vessel_1', t='Timestamp')
#vessel_log.to_line_gdf()
vessel_log.to_line_gdf().to_file('plots_routes/wegafzetting_logboek_movingpandas.gpkg')

d:\Users\ALPF\AppData\Local\pypoetry\Cache\virtualenvs\dtv-backend-pvs5eNot-py3.10\lib\site-packages\movingpandas\trajectory.py:166: MissingCRSWarning:

Trajectory generated without CRS. Computations will use Euclidean distances.



In [19]:
vessels[0].logbook


[{'Message': 'Start sailing',
  'Timestamp': datetime.datetime(2024, 6, 3, 14, 23, 10),
  'Value': '',
  'Geometry': <POINT (4.633 51.783)>},
 {'Message': 'Sailing to start',
  'Timestamp': datetime.datetime(2024, 6, 4, 4, 7, 15, 180390),
  'Value': 49445.180389901776,
  'Geometry': <POINT (5.349 51.804)>},
 {'Message': 'Sailing from node 8865735 to node 22638402 sub edge 0 start',
  'Timestamp': datetime.datetime(2024, 6, 4, 4, 7, 15, 180390),
  'Value': 0,
  'Geometry': <POINT (5.349 51.804)>},
 {'Message': 'Sailing from node 8865735 to node 22638402 sub edge 0 stop',
  'Timestamp': datetime.datetime(2024, 6, 4, 4, 10, 9, 283271),
  'Value': 0,
  'Geometry': <POINT (5.351 51.803)>},
 {'Message': 'Sailing from node 22638402 to node B51956_B sub edge 0 start',
  'Timestamp': datetime.datetime(2024, 6, 4, 4, 10, 9, 283271),
  'Value': 0,
  'Geometry': <POINT (5.351 51.803)>},
 {'Message': 'Sailing from node 22638402 to node B51956_B sub edge 0 stop',
  'Timestamp': datetime.datetime(202

### Visualization of path
If you get an error regarding ffmpeg use [this](https://stackoverflow.com/questions/13316397/matplotlib-animation-no-moviewriters-available) answer. You have to install ffmpeg in your Conda environment. It can be done using the following command.

```bash
#Install ffmpeg using Conda
conda install -c conda-forge ffmpeg
```

In [20]:
# visualise vessel movements based on the information included in the vessel.log
# plot.vessel_kml(env, vessels, stepsize = 60)
#plot.graph_kml(env)


In [21]:
! start explorer .